In [1]:
from sklearn.preprocessing import LabelEncoder

import pandas as pd

from sklearn.ensemble import RandomForestClassifier



In [2]:
veri = {
    "Yaş":[25,30,35,20,40],
    "Film_Tercihi":["Aksiyon","Komedi","Bilim Kurgu","Romantik","Komedi"],
    "Son_Izlenen":["Komedi","Bilim Kurgu","Aksiyon","Romantik","Romantik"],
    "Dusunulen_Film":["Aksiyon","Romantik","Bilim Kurgu","Komedi","Aksiyon"],
    "Izleme_Durumu":["Izler","Izler","Izlemez","Izler","Izlemez"]
}

df = pd.DataFrame(veri)
df.head()

,Yaş,Film_Tercihi,Son_Izlenen,Dusunulen_Film,Izleme_Durumu
0,25,Aksiyon,Komedi,Aksiyon,Izler
1,30,Komedi,Bilim Kurgu,Romantik,Izler
2,35,Bilim Kurgu,Aksiyon,Bilim Kurgu,Izlemez
3,20,Romantik,Romantik,Komedi,Izler
4,40,Komedi,Romantik,Aksiyon,Izlemez


In [3]:
from re import X
le = LabelEncoder()
# fit_transform = iki adımlı dönüşüm yapar;
#ilk adımda dönüşüm yapılacak parametreyi öğrenir(fit)
#ikinci adımda dönüşümü yapar(transform).
df["Film_Tercihi"]= le.fit_transform(df["Film_Tercihi"])
df["Son_Izlenen"]= le.fit_transform(df["Son_Izlenen"])
df["Dusunulen_Film"]= le.fit_transform(df["Dusunulen_Film"])

# Bağımlı ve B.sız değişkenleri ayırma

X=df[["Yaş","Dusunulen_Film","Son_Izlenen","Film_Tercihi"]]
y=df["Izleme_Durumu"]


model = RandomForestClassifier(n_estimators=100,random_state=42)
model.fit(X,y)

yeni_kullanici = [[32,le.transform(["Romantik"])[0],
                   le.transform(["Aksiyon"])[0],
                   le.transform(["Komedi"])[0]]]

tahmin = model.predict(yeni_kullanici)

print("Kullanıcının film izleme durumu:", tahmin)

Kullanıcının film izleme durumu: ['Izler']


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Random Forest ve Decision Tree Farkları

1. Temel Yapı = DT karar düğümleri üzerinden karar verir; RF birden fazla DT bir araya gelerek oluşur ve karar verilir.

2. Overfitting = DT overfitting yaşanabilir. Eğitim verilerine uygun ancak genelleme yeteneği düşük bir model oluşabilir. RF birden fazla DT bir araya gelerek oluştuğu için her bir DT overfitting eğilimi gösterse bile genelleme yeteneği yüksektir.

3. Tahmin Gücü = RF DT lerden bir araya geldiği için tahminlerde daha güçlüdür.

4. Hiperparametre Ayarları = DT nin çok fazla hiperparametresi yoktur bundan dolayı daha kolay ayarlanabilir.RF çok fazla hiperparametreye sahip olduğu için modelin karmaşıklığını arttırabilir.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [6]:
df = pd.read_csv('Churn.csv')
df.head()

,Customer ID,Gender,Senior Citizen,Partner,Dependents,tenure,Phone Service,Multiple Lines,Internet Service,Online Security,...,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
0,7590-VHVEA,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
2,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
3,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
4,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No


In [18]:
X= df.drop(["Churn","Customer ID"],axis=1) # drop() dataframe den belirtilen sütunu kaldırır(bırakır),axis=1 işlemin sütunlarda yapılacağını gösterir.
y= df['Churn'] # hedef değişken

le = LabelEncoder()
for col in X.select_dtypes(include=["object"]).columns:
  X[col] = le.fit_transform(X[col])




le = LabelEncoder()
X['Gender'] = le.fit_transform(X['Gender'])
X['Partner'] = le.fit_transform(X['Partner'])
X['Dependents'] = le.fit_transform(X['Dependents'])
columns_to_be_encoded = ['Gender', 'Senior Citizen']  # list of column names you want encoded
# Instantiate the encoders
encoders = {column: LabelEncoder() for column in columns_to_be_encoded}
for column in columns_to_be_encoded:
    df[column] = encoders[column].fit_transform(df[column])

In [20]:
#Eğitim ve test veri setlerine ayırma
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=42)
# model oluşturma
model = RandomForestClassifier(n_estimators=100,random_state=42)
#model eğitme
model.fit(X_train, y_train)
# Test kümesi üzerinde tahmin yapma
y_pred = model.predict(X_test)

#modelin performansını değerlendirme

accuracy = accuracy_score(y_test,y_pred) # modelin doğru sınıflandırılan örneklerinin toplam örnek sayısına oranı

confmat = confusion_matrix(y_test,y_pred) # modelin sınıflandırma yaparken ne kadar doğru ne kadar yanlış yaptığını gösteren metrik.
# her satır gerçek sınıfları;her sütun tahmin edilen sütunları ifade eder.

classreport = classification_report(y_test,y_pred) # her bir sınıf için doğruluk, hassasiyet, geri çağırma, ve F1 Score değerlerini sunar.


print("Accuracy :", accuracy)
print("Confusion MAtrix :\n", confmat)
print("Sınıflandırma Raporu :\n", classreport)

Accuracy : 0.7934705464868701
Confusion MAtrix :
 [[938 102]
 [189 180]]
Sınıflandırma Raporu :
               precision    recall  f1-score   support

          No       0.83      0.90      0.87      1040
         Yes       0.64      0.49      0.55       369

    accuracy                           0.79      1409
   macro avg       0.74      0.69      0.71      1409
weighted avg       0.78      0.79      0.78      1409

